In [1]:
import torch, yaml
from ultralytics import YOLO
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile
from torch.utils.data import DataLoader
from torchvision.transforms import v2

import os
import onnx
import numpy as np

import sys
root_dir = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(root_dir)


from cnn.hands_cnn import Hands_VGG16
from cnn.face_cnn import Face_CNN
from cnn.raw_cnn import Raw_CNN
from cnn.dataset import V2Dataset

from wrappers.face_wrapper import Face_Inference_Wrapper
from wrappers.hands_wrapper import Hands_Inference_Wrapper


from ensemble import Ensemble_Inference

In [2]:
hands_path = (r"/home/ron/Classes/CV-Systems/cybertruck/detection/hands_detection/runs/detect/best/weights/best.pt")  # load a custom trained model
face_path = (r"/home/ron/Classes/CV-Systems/cybertruck/detection/face_detection/weights/yolov8n-face.pt")  # load a custom trained model

class HandArgs:
    def __init__(self):
        self.freeze = True
        self.num_frozen_params = 30
        self.dropout = 0.35
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

hand_args = HandArgs()

In [3]:
test_transform = v2.Compose([
        v2.ToPILImage(),
        v2.Resize((299,299)),
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.3879, 0.3879, 0.3879], std=[0.3001, 0.3001, 0.3001])
    ])

val_dataset = V2Dataset(cam1_path=f'/home/ron/Classes/CV-Systems/cybertruck/data/v2_cam1_cam2_split_by_driver/Camera 1/test', cam2_path=f'/home/ron/Classes/CV-Systems/cybertruck/data/v2_cam1_cam2_split_by_driver/Camera 2/test', transform=test_transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [4]:

face_model = Face_CNN(None).to('cuda')
x = torch.randn(5, 3, 299, 299).to('cuda')
face_model.load_state_dict(torch.load(r"/home/ron/Classes/CV-Systems/cybertruck/cnn/face_models/face/SGD/epoch10_11-28_10:50:06_66acc.pt"))
face_wrapper = Face_Inference_Wrapper(face_model, face_path)

hands_model = Hands_VGG16(hand_args).to('cuda')
x = torch.randn(5, 3, 224, 224).to('cuda')
hands_model.load_state_dict(torch.load(r"/home/ron/Classes/CV-Systems/cybertruck/cnn/hands_models/vgg/epoch60_11-16_03:44:44.pt"))
hands_wrapper = Hands_Inference_Wrapper(hands_model, hands_path)

raw_model = Raw_CNN(None).to('cuda')
x = torch.randn(5, 3, 299, 299).to('cuda')
raw_model.load_state_dict(torch.load(r"/home/ron/Classes/CV-Systems/cybertruck/cnn/raw_models/raw/SGD/epoch20_11-27_16:15:10_76acc.pt"))
raw_model.eval()

model_list = [face_wrapper, hands_wrapper, raw_model]

ensemble_model = Ensemble_Inference(hand_args, model_list, val_loader, num_classes=10)

/home/ron/miniconda3/envs/cyber/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


In [5]:
scripted_module = torch.jit.script(ensemble_model)
torchscript_model_optimized = optimize_for_mobile(scripted_module)
# Export lite interpreter version model (compatible with lite interpreter)
scripted_module._save_for_lite_interpreter("Ensemble.ptl")

NotSupportedError: Compiled functions can't take variable number of arguments or use keyword-only arguments with defaults:
  File "/home/ron/miniconda3/envs/cyber/lib/python3.10/site-packages/ultralytics/nn/tasks.py", line 30
    def forward(self, x, *args, **kwargs):
                                 ~~~~~~~ <--- HERE
        """
        Forward pass of the model on a single scale. Wrapper for `_forward_once` method.
